# PTCB project

##### TODO
- Kallisto : Quantification des transcrits.


- QC step after mapping => rseqc (http://rseqc.sourceforge.net/).


- Comparaison avec résultats de Nice.


- Justifier l'utilisation de DESeq2 pour les cas sans réplicats : <br>
Pourquoi lui plutot que par exemple EdgeR ? <br>
Ecrire noir sur blanc les limitations des analyses 1vs1.


- Justifier / vérifier que les comptes sont bien strand-specific : <br>
Soit trouver littérature qui dit que featureCounts, à partir de l'annotation (GENCODE), ne compte que les paires de reads qui ont pour brin originel le brin +/- indiqué pour chaque gène. Trouver également littérature qui justifie pour Kallisto (on sait déjà pour ces deux outils qu'il y a des paramètres de strand-spécificité). <br>
Soit vérifier un cas "à la main", cad trouver dans un dataset un endroit où il y a profondeur de reads sur chaque brin, et alors vérifier que les comptes sont bons.


- Exploration des transcrits anti-sens : <br>
Finir de setup Sandrine pour utilisation d'IGV (outil sur son PC + load bigwig et annotation GENCODE). <br>
Ajouter annotation manquantes "à la main" (custom track d'annotation) sur le locus IgH quand Sandrine nous les aura fourni. <br>
Tester le pipeline UClnrR pour une exploration systématique : http://bioinformaticstools.mayo.edu/research/uclncr-pipeline.

##### Notes

Report :
http://unil.im/Ly33D

Normalization :
http://unil.im/kdTj5

FeatureCounts tests :
http://unil.im/2sAq6

Read counts step by step :
http://unil.im/Ic1Wc

Pefanis : Correspond à du stimulé.

Réplicats (incoming) : 1 R "Eµ FL KO" (replicat de Yves EU), 1 R MAR Ko (replicat de Yves) & 2 R cEµ KO.

Histoire de recomb/transloc random qui peuvent générer des reads en anti-sens sur le génome => need de prendre le BAM, de soustraire tous les reads qui sont sur un locus : "gene pos pos strand" connu, puis de regarder ce qu'il reste.

## 1) Data

### 1.1) Raw reads

#### 1.1.1) CRIBL data

Stored @Sophia Antipolis : <br>
Eric : 3B4, 3pRR, DEL, IRIS, PAL, WT. <br>
Yves : 3pRR, EU, MAR, WT. <br>
Replicates (incoming) : EU-R (~Yves), MAR-R (~Yves), 2 CEU-R (new).

In [ ]:
### CODE ###
mkdir -p ~/Desktop/PTCB/BISCEm/Data/Reads_raw_Nice;
cd ~/Desktop/PTCB/BISCEm/Data/Reads_raw_Nice;

In [ ]:
### CODE ###
axel -q http://www.genomique.info/data/public/000-export/151109_pinaud.tar;
axel -q http://www.genomique.info/data/public/000-export/DEL_IRIS_fastq.tar;
axel -q http://www.genomique.info/data/public/000-export/160627_denizot.tar;
tar -xvf *.tar;

And some files renaming (for which I lost the script).

#### 1.1.2) Pefanis data

Retrieve all "BC" samples : https://www.ncbi.nlm.nih.gov/sra?term=SRP042355 :

In [ ]:
### CODE ###
for id in 'SRR1302208' 'SRR1302283' 'SRR1302414' \
           'SRR1302785' 'SRR1302786' 'SRR1302787' \
           'SRR1302788' 'SRR1302789' 'SRR1984164' \
           'SRR1984166' 'SRR1984167' 'SRR1984178';
do fastq-dump -O . --split-files $id;
done;

### 1.2) Mus musculus

#### 1.2.1) Reference genome & associated GFF

Downloaded from the "Genome Reference Consortium", latesst realease = GRCm38.p5 :

In [ ]:
### CODE ###
mkdir -p ~/Desktop/PTCB/BISCEm/Data/Mus_musculus;
cd ~/Desktop/PTCB/BISCEm/Data/Mus_musculus;

In [ ]:
### CODE ###
url='ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000/001/635/GCA_000001635.7_GRCm38.p5';
axel -q $url/GCA_000001635.7_GRCm38.p5_genomic.fna.gz;
wget -nv $url/GCA_000001635.7_GRCm38.p5_genomic.gff.gz;
unpigz *_GRCm38.p5_genomic*;

Check if they contains the same sequences names :

In [ ]:
### CODE ###
grep '^>' GCA_000001635.7_GRCm38.p5_genomic.fna | sort -u > fna.txt
sed -i 's/>//' fna.txt 
awk -F "[[:space:]]" '{print $1}' fna.txt > fna_final.txt
awk -F "\t" '{if($0 !~ /^#/) print $1}' GCA_000001635.7_GRCm38.p5_genomic.gff | sort -u > gff.txt
diff fna_final.txt gff.txt

If the diff command produce no output, it's okay. <br>
One last thing, to avoid potential incompatilities in downstream analysis :

In [ ]:
### CODE ###
grep '^>' GCA_000001635.7_GRCm38.p5_genomic.fna | head -2;
awk -F "\t" '{if($0 !~ /^#/) print $1}' GCA_000001635.7_GRCm38.p5_genomic.gff | head -2;

What we have just shown is that sequences headers in the FNA file are more verbose than the associated name for the same sequence in the GFF file. Given that this extra info is not critical, let's discard it & check that we did this successfully :

In [ ]:
### CODE ###
gawk -i inplace -F "[[:space:]]" '{if($0 ~ /^>/) $0 = $1; print}' GCA_000001635.7_GRCm38.p5_genomic.fna;
grep '^>' GCA_000001635.7_GRCm38.p5_genomic.fna | head -2;
awk -F "\t" '{if($0 !~ /^#/) print $1}' GCA_000001635.7_GRCm38.p5_genomic.gff | head -2;

Load stuff on Genotoul :

In [ ]:
### CODE ###
sudo scp *_GRCm38.p5_genomic* escaon@genotoul.toulouse.inra.fr:/home/escaon/work/PTCB/Mus_musculus;

Remove useless files & compress the remaining for local storage :

In [ ]:
### CODE ###
rm *.txt;
pigz *_GRCm38.p5_genomic*;

#### 1.2.2) Reference transcriptome

Which refererence transcriptome to use with Kallisto ? <br>

Asked question : I plan to work with Kallisto on the mus musculus transcriptome : Which DB should I use to retrieve the transcriptome ? : Genbank, Ensembl or UCSC ? <br>

Answer (Kallisto authors) : "Anyway, our links would simply point you to the ensembl transcriptome when it exists. We typically use this as it is fairly complete (by comparison with others) and the transcript to gene name mapping is pretty nice (also by comparison with others)." <br>

Thus we retrieve transcriptome from Ensembl latest release and use cdna + ncrna (to covern almost as many genes as the gencode annotation used with featureCounts). <br>

In [ ]:
### CODE ###
mkdir -p /media/sf_raid/Data/PTCB/mm10_transcriptome;
cd /media/sf_raid/Data/PTCB/mm10_transcriptome;

In [ ]:
### CODE ###
url='ftp://ftp.ensembl.org/pub/release-90/fasta/mus_musculus';
axel -q $url/cdna/Mus_musculus.GRCm38.cdna.all.fa.gz;
axel -q $url/ncrna/Mus_musculus.GRCm38.ncrna.fa.gz;
cat Mus_musculus.GRCm38.cdna.all.fa.gz \
    Mus_musculus.GRCm38.ncrna.fa.gz \
    > mus_musculus_ensembl_grcm38_r90_cdna_and_ncrna.fasta.gz;

Load stuff on Genotoul :

In [ ]:
### CODE ###
sudo scp mus_musculus_ensembl_grcm38_r90_cdna_and_ncrna.fasta.gz \
     hbastien@genotoul.toulouse.inra.fr:/home/hbastien/work/PTCB/mm10_transcriptome;

#### 1.2.3) GTF annotations

For future use with featureCounts.<br>

Downloaded from "GENCODE" (for GRCm38.p5 genome). <br>
"It contains the comprehensive gene annotation on the reference chromosomes, scaffolds, assembly patches and alternate loci (haplotypes)". <br>
More precisely, it annotate a lot of gene/transcript biotypes : https://www.gencodegenes.org/gencode_biotypes.html.

In [ ]:
### CODE ###
axel -q ftp://ftp.sanger.ac.uk/pub/gencode/Gencode_mouse/\
release_M14/gencode.vM14.chr_patch_hapl_scaff.annotation.gtf.gz;

#### 1.2.4) Ensembl vs GENCODE

We want at some point to compare results obtained with the "genomic path", i.e. STAR + featureCounts + DESeq2 vs results from the "transcriptomic path", i.e. Kallisto + DESeq2. Thus, we need make sure that featureCounts (using GENCODE annotations) and Kallisto (using transcriptomes sequences) will compute reads counts at the gene level for approximately the same genes. <br>

##### Get Ensembl genes name list contained in GENCODE gtf : 

In [ ]:
### CODE ###
cd ~/Desktop/PTCB/BISCEm/Data/Mus_musculus;
unpigz gencode.vM14.chr_patch_hapl_scaff.annotation.gtf.gz;
awk -F " " '{if($0 !~ /^#/) print $10}' gencode.vM14.chr_patch_hapl_scaff.annotation.gtf | sort -u > gencode_gene_ids_u.txt;
sed -i 's/;//g' gencode_gene_ids_u.txt;
sed -i 's/\"//g' gencode_gene_ids_u.txt;
awk -F '\t' -v OFS='\t' '{sub(/\.[0-9]*/, "", $1)} 1' gencode_gene_ids_u.txt > gencode_gene_ids_u_proper.txt;
wc -l gencode_gene_ids_u_proper.txt;
# 52366 gencode_gene_ids_u_proper.txt
rm gencode_gene_ids_u.txt;
pigz gencode.vM14.chr_patch_hapl_scaff.annotation.gtf;
cp gencode_gene_ids_u_proper.txt ~/Desktop/PTCB/BISCEm/Ensembl_transcriptome_vs_gencode;

##### Get Ensembl genes name list contained in our Ensembl transcriptome :

In [ ]:
### CODE ###
cd ~/Desktop/PTCB/BISCEm/Ensembl_transcriptome_vs_gencode;
# DL
axel -q ftp://ftp.ensembl.org/pub/release-90/fasta/mus_musculus/cdna/Mus_musculus.GRCm38.cdna.all.fa.gz;
axel -q ftp://ftp.ensembl.org/pub/release-90/fasta/mus_musculus/ncrna/Mus_musculus.GRCm38.ncrna.fa.gz;
# Grep non-fasta lines
zgrep '^>' Mus_musculus.GRCm38.cdna.all.fa.gz > headers_cdna.txt;
zgrep '^>' Mus_musculus.GRCm38.ncrna.fa.gz > headers_ncrna.txt;
# Count number of transcripts
for f in *.txt;
do printf $f"\t";
   sort -u $f | wc -l;
done;
# headers_cdna.txt    109282
# headers_ncrna.txt    19932
# Clear formatting -> ensembl_gene_name
awk -F " " '{print $4}' headers_cdna.txt > genes_cdna.txt;
awk -F " " '{print $4}' headers_ncrna.txt > genes_ncrna.txt;
sort -u genes_cdna.txt > genes_cdna_u.txt;
sort -u genes_ncrna.txt > genes_ncrna_u.txt;
awk -F "\t" '{sub(/\.[0-9]*/, "", $1)} 1' genes_cdna_u.txt > genes_cdna_u_v2.txt;
awk -F "\t" '{sub(/\.[0-9]*/, "", $1)} 1' genes_ncrna_u.txt > genes_ncrna_u_v2.txt;
awk -F "\t" '{sub(/gene:/, "", $1)} 1' genes_cdna_u_v2.txt > genes_cdna_u_v3.txt;
awk -F "\t" '{sub(/gene:/, "", $1)} 1' genes_ncrna_u_v2.txt > genes_ncrna_u_v3.txt;
wc -l *_v3.txt;
# 34839 genes_cdna_u_v3.txt
# 15220 genes_ncrna_u_v3.txt
# 50059 total
cat genes_cdna_u_v3.txt genes_ncrna_u_v3.txt > genes_cdna_and_ncrna.txt;
sort -u genes_cdna_and_ncrna.txt > genes_cdna_and_ncrna_u.txt;
wc -l genes_cdna_and_ncrna_u.txt;
# 50059 genes_cdna_and_ncrna_u.txt;
rm !(gencode_gene_ids_u_proper.txt|genes_cdna_and_ncrna_u.txt);

##### Finally, check their intersection :

In [ ]:
### CODE ###
comm gencode_gene_ids_u_proper.txt genes_cdna_and_ncrna_u.txt > comm_gencode_vs_ensembl.tsv;
printf "unique to gencode :\t"; awk -F "\t" '{if($1 != "") print $1}' comm_gencode_vs_ensembl.tsv | wc -l;
printf "unique to ensembl :\t"; awk -F "\t" '{if($2 != "") print $2}' comm_gencode_vs_ensembl.tsv | wc -l;
printf "common to both :\t"; awk -F "\t" '{if($3 != "") print $3}' comm_gencode_vs_ensembl.tsv | wc -l;
# unique to gencode :    3047
# unique to ensembl :    740
# common to both :    49319

49319 genes are common (92.87%), so we consider that we can compare DE results with counts coming from featureCounts and from Kallisto.

## 2) Raw reads QC & filtering

### 2.1) FastQC

#### 2.1.1) CRIBL data

Setup working directory & paths :

In [ ]:
### CODE ###
reads_dir='/home/erwann/Desktop/PTCB/BISCEm/Data/Reads_raw_Nice';
cd $reads_dir;
mkdir -p $reads_dir/QC;
mkdir -p ~/Software/FastQC/Outputs/PTCB;

Launch FastQC :

In [ ]:
### CODE ###
for f in *.fastq.gz;
do echo $f; fastqc --outdir ~/Software/FastQC/Outputs/PTCB -f fastq $f;
done >> $reads_dir/QC/fastqc.verbose;

#### 2.1.2) Pefanis data

The exact same thing, just change file and directory names

### 2.2) MultiQC

#### 2.2.1) CRIBL data

We compile FastqQC analysis for all the data & at the sequencing-run level (EP & YD), to highlight potential run effect.

In [ ]:
### CODE ###
fastqc_dir='/home/erwann/Software/FastQC/Outputs/PTCB';
mkdir -p $reads_dir/QC;
multiqc $fastqc_dir \
        -n ptcb \
        -o $reads_dir/QC \
        -m fastqc -f > $reads_dir/QC/multiQC_ptcb.verbose 2>&1;
multiqc $fastqc_dir/*_EP_fastqc* \
        -n ptcb_ep \
        -o $reads_dir/QC \
        -m fastqc -f > $reads_dir/QC/multiQC_ptcb_ep.verbose 2>&1;
multiqc $fastqc_dir/*_YD_fastqc* \
        -n ptcb_yd \
        -o $reads_dir/QC \
        -m fastqc -f > $reads_dir/QC/multiQC_ptcb_yd.verbose 2>&1;

Visualize QC results :

In [ ]:
### CODE ###
chromium-browser $reads_dir/QC/*.html;

Conclusions :
Nice data overall. We can be a bit stringent during filtering to keep HQ reads.

#### 2.2.2) Pefanis data

The exact same thing, just change file and directory names

### 2.3) Filtering

Trimmomatic 0.36

For both data CRIBL & Pefanis, FastQC detected Illumina 1.9 encoding. Referring to https://en.wikipedia.org/wiki/FASTQ_format, Illumina 1.8+ => Phred+33.

Sequences are trimmed with $TRIM = ~/Software/Trimmomatic_0.36/trimmomatic-0.36.jar.
Given MultiQC output, we trim starting and ending bases if their quality is < 25. We then trim with a sliding window of 5 bases, cutting the read at those bases if the mean quality drops below 25. Finally, we discard sequence < 50 nts.

#### 2.3.1) CRIBL data

In [ ]:
### CODE ###
$TRIM = ~/Software/Trimmomatic_0.36/trimmomatic-0.36.jar
cd ~/Desktop/PTCB/BISCEm/Data/Reads_raw_Nice;
mkdir -p ../Reads_raw_Nice_trim;
for id in '3B4_R_1_EP' '3B4_S_1_EP' '3pRR_R_1_EP' '3pRR_S_1_EP' \
          'DEL_R_1_EP' 'DEL_S_1_EP' 'IRIS_R_1_EP' 'IRIS_S_1_EP' \
          'PAL_R_1_EP' 'PAL_S_1_EP' 'WT_R_1_EP' 'WT_S_1_EP';
do java -jar $TRIM PE -threads 8 \
                      -phred33 \
                      $id.fastq.gz \
                      ${id%_*_*}'_2_EP.fastq.gz' \
                      ../Reads_raw_Nice_trim/$id'_paired.fastq.gz' \
                      ../Reads_raw_Nice_trim/$id'_unpaired.fastq.gz' \
                      ../Reads_raw_Nice_trim/${id%_*_*}'_2_EP_paired.fastq.gz' \
                      ../Reads_raw_Nice_trim/${id%_*_*}'_2_EP_unpaired.fastq.gz' \
                      LEADING:25 TRAILING:25 SLIDINGWINDOW:5:25 MINLEN:50;
done &>> trim_EP.verbose;

for id in '3pRR_R_1_YD' '3pRR_S_1_YD' 'EU_R_1_YD' 'EU_S_1_YD' \
          'MAR_R_1_YD' 'MAR_S_1_YD' 'WT_R_1_YD' 'WT_S_1_YD';
do java -jar $TRIM PE -threads 8 \
                      -phred33 \
                      $id.fastq.gz \
                      ${id%_*_*}'_2_YD.fastq.gz' \
                      ../Reads_raw_Nice_trim/$id'_paired.fastq.gz' \
                      ../Reads_raw_Nice_trim/$id'_unpaired.fastq.gz' \
                      ../Reads_raw_Nice_trim/${id%_*_*}'_2_YD_paired.fastq.gz' \
                      ../Reads_raw_Nice_trim/${id%_*_*}'_2_YD_unpaired.fastq.gz' \
                      LEADING:25 TRAILING:25 SLIDINGWINDOW:5:25 MINLEN:50;
done &>> trim_YD.verbose;

#### 2.3.2) Pefanis data

In [ ]:
### CODE ###
$TRIM = ~/Software/Trimmomatic_0.36/trimmomatic-0.36.jar
cd /media/raid/Data/PTCB/Pefanis;
mkdir -p Trimmomatic;
for id in 'SRR1302208' 'SRR1302283' 'SRR1302786' 'SRR1302787' \
          'SRR1302788' 'SRR1302789';
do java -jar $TRIM PE -threads 8 \
                      -phred33 \
                      $id'_1.fastq' \
                      $id'_2.fastq' \
                      ./Trimmomatic/$id'_1.paired.fastq' \
                      ./Trimmomatic/$id'_1.unpaired.fastq' \
                      ./Trimmomatic/$id'_2.paired.fastq' \
                      ./Trimmomatic/$id'_2.unpaired.fastq' \
                      LEADING:25 TRAILING:25 SLIDINGWINDOW:5:25 MINLEN:50;
done &>> trim_PE.verbose;
for id in 'SRR1302414' 'SRR1302785';
do java -jar $TRIM SE -threads 8 \
                      -phred33 \
                      $id'_1.fastq' \
                      ./Trimmomatic/$id'_1.paired.fastq' \
                      ./Trimmomatic/$id'_1.unpaired.fastq' \
                      LEADING:25 TRAILING:25 SLIDINGWINDOW:5:25 MINLEN:50;
done &>> trim_SE.verbose;

## 3) Mapping

### 3.1) Build index :

#### 3.1.1) STAR :

On Genotoul via STAR, version 2.5.2b :

In [ ]:
### CODE ###
#!/bin/bash
#$ -q workq
#$ -M erwan.scaon@unilim.fr
#$ -m bea
#$ -l mem=8G
#$ -l h_vmem=12G
#$ -N star_idx
#$ -o /home/escaon/work/PTCB/star_idx.o
#$ -e /home/escaon/work/PTCB/star_idx.e

work='/home/escaon/work/PTCB/Mus_musculus';
star='/usr/local/bioinfo/src/STAR/STAR-2.5.2b/bin/Linux_x86_64_static/STAR'
nt=8;

$star --runThreadN $nt \
      --runMode genomeGenerate \
      --genomeDir $work \
      --genomeFastaFiles $work/GCA_000001635.7_GRCm38.p5_genomic.fna \
      --sjdbGTFfile $work/GCA_000001635.7_GRCm38.p5_genomic.gff \
      --sjdbGTFtagExonParentTranscript Parent \
      --sjdbOverhang 150;

In [ ]:
### CODE ###
qsub -pe parallel_smp 8 star_idx.sh;

#### 3.1.2) Kallisto :

On Genotoul via Kallisto, version 0.43.0 :

In [ ]:
### CODE ###
#!/bin/bash
#$ -q workq
#$ -m bea
#$ -l mem=8G
#$ -l h_vmem=120G

#####Index KALLISTO
kallisto index --make-unique -i /home/hbastien/work/PTCB/mm10_transcriptome/Mus_musculus.GRCm38.cdna.all.idx /home/hbastien/work/PTCB/mm10_transcriptome/Mus_musculus.GRCm38.cdna.all.fa

In [ ]:
### CODE ###
qsub simple_index_kallisto.sh;

### 3.2) Map reads :

###### 3.2.1) CRIBL data

First script : star_3B4_R_EP.sh

In [ ]:
### CODE ###
#!/bin/bash
#$ -q workq
#$ -M erwan.scaon@unilim.fr
#$ -m bea
#$ -l mem=8G
#$ -l h_vmem=10G
#$ -N star_3B4_R_EP
#$ -o /home/escaon/work/PTCB/star_3B4_R_EP.o
#$ -e /home/escaon/work/PTCB/star_3B4_R_EP.e

star='/usr/local/bioinfo/src/STAR/STAR-2.5.2b/bin/Linux_x86_64_static/STAR';
save='/home/escaon/save/PTCB/Reads_raw_Nice_trim';
work='/home/escaon/work/PTCB';
nt=8;

for id in '3B4_R_EP';
do $star --runThreadN $nt \
         --genomeDir $work/Mus_musculus \
         --readFilesIn $save/${id%_*}'_1_EP_paired.fastq.gz' $save/${id%_*}'_2_EP_paired.fastq.gz' \
         --readFilesCommand zcat \
         --outFileNamePrefix $work/Mapped_vs_GRCm38.p5/$id'_' \
         --outSAMtype BAM SortedByCoordinate;
done;

In [ ]:
### CODE ###
qsub -pe parallel_smp 8 star_3B4_R_EP.sh;

Duplicate this script for others EP datasets & launch jobs :

In [ ]:
### CODE ###
for id in '3B4_S_EP' '3pRR_R_EP' '3pRR_S_EP' 'DEL_R_EP' \
          'DEL_S_EP' 'IRIS_R_EP' 'IRIS_S_EP' 'PAL_R_EP' \
          'PAL_S_EP' 'WT_R_EP' 'WT_S_EP';
do  cp star_3B4_R_EP.sh star_$id.sh;
    sed -i "s/3B4_R_EP/$id/" star_$id.sh;
    qsub -pe parallel_smp 8 star_$id.sh;
done;

Create a base script for YD, duplicate it & launch jobs :

In [ ]:
### CODE ###
cp star_WT_R_EP.sh star_WT_R_YD.sh;
grep --color 'EP' star_WT_R_YD.sh; # check if we are not going to destroy anything
sed -i "s/EP/YD/" star_WT_R_YD.sh;
qsub -pe parallel_smp 8 star_WT_R_YD.sh;

for id in '3pRR_R_YD' '3pRR_S_YD' 'EU_R_YD' 'EU_S_YD' \
          'MAR_R_YD' 'MAR_S_YD' 'WT_S_YD';
do  cp star_WT_R_YD.sh star_$id.sh;
    sed -i "s/WT_R_YD/$id/" star_$id.sh;
    qsub -pe parallel_smp 8 star_$id.sh;
done;

###### 3.2.2) Pefanis data

###### Paired end :

In [ ]:
### CODE ###
#!/bin/bash
#$ -q workq
#$ -m bea
#$ -l mem=8G
#$ -l h_vmem=30G
#

#####Alignment STAR PE
for id in 'SRR1302208' 'SRR1302283' 'SRR1302786' 'SRR1302787' 'SRR1302788' 'SRR1302789';
do
/usr/local/bioinfo/src/STAR/STAR-2.5.2b/bin/Linux_x86_64_static/STAR --genomeDir /home/hbastien/work/PTCB/Mus_musculus/ --readFilesIn /home/hbastien/work/PTCB/Pefanis/'Trimmed_paired_'$id'_1.fastq' /home/hbastien/work/PTCB/Pefanis/'Trimmed_paired_'$id'_2.fastq' --outFileNamePrefix Experiment_PE_Star$id
done

In [ ]:
### CODE ###
qsub manageStarPE.sh

###### Single end :

In [ ]:
### CODE ###
#!/bin/bash
#$ -q workq
#$ -m bea
#$ -l mem=8G
#$ -l h_vmem=30G
#

#####Alignment STAR SE
for id in 'SRR1302414' 'SRR1302785';
do
/usr/local/bioinfo/src/STAR/STAR-2.5.2b/bin/Linux_x86_64_static/STAR --genomeDir /home/hbastien/work/PTCB/Mus_musculus/ --readFilesIn /home/hbastien/work/PTCB/Pefanis/'Trimmed_'$id'_1.fastq' --outFileNamePrefix Experiment_SE_Star$id
done

In [ ]:
### CODE ###
qsub manageStarSE.sh

### 3.3) Index BAM

In [ ]:
### CODE ###
#!/bin/bash
#$ -q workq
#$ -M erwan.scaon@unilim.fr
#$ -m bea
#$ -l mem=40G
#$ -l h_vmem=48G
#$ -N samtools_idx
#$ -o /home/escaon/work/PTCB/samtools_idx.o
#$ -e /home/escaon/work/PTCB/samtools_idx.e

work='/home/escaon/work/PTCB/Mapped_vs_GRCm38.p5';

cd $work;
for f in *.bam;
do samtools index $f;
done;

In [ ]:
### CODE ###
qsub samtools_idx.sh;

### 3.4) RNA-seq alignment QC

http://rseqc.sourceforge.net

## 4) Counts

##### Retrieve BAMs locally :

In [ ]:
### CODE ###
mkdir -p ~/Desktop/PTCB/BISCEm/Outputs;
sudo scp -r escaon@genotoul.toulouse.inra.fr:/home/escaon/work/PTCB/Mapped_vs_GRCm38.p5 ~/Desktop/PTCB/BISCEm/Outputs;
sudo scp -r hbastien@genotoul.toulouse.inra.fr:/home/hbastien/work/PTCB/Mapped_vs_GRCm38.p5 /run/user/1000/gvfs/smb-share:server=ubox.unilim.fr,share=biscem/BioInformatique/Responsables/Commun/PTCB/Outputs/Star

Before starting the counts, we need to create a "chr.aliases" file which link chrs names in GRCm38.p5 with chrs names in GENCODE annotations. This file was created from this assembly report :

In [ ]:
### CODE ###
wget -q ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000/001/635/\
GCA_000001635.7_GRCm38.p5/GCA_000001635.7_GRCm38.p5_assembly_report.txt;

### 4.1) FeatureCounts

Count with featureCounts (Rsubread 1.24.2) :

#### 4.1.1) CRIBL data

In [ ]:
### CODE ###
path='/home/erwann/Desktop/PTCB/BISCEm';
cd $path/Outputs/Star;
for id in '3B4_R_EP' '3B4_S_EP' '3pRR_R_EP' '3pRR_S_EP' \
          'DEL_R_EP' 'DEL_S_EP' 'IRIS_R_EP' 'IRIS_S_EP' \
          'PAL_R_EP' 'PAL_S_EP' 'WT_R_EP' 'WT_S_EP' \
          '3pRR_R_YD' '3pRR_S_YD' 'EU_R_YD' 'EU_S_YD' \
          'MAR_R_YD' 'MAR_S_YD' 'WT_R_YD' 'WT_S_YD';
do Rscript $path/Scripts/featureCounts.R $id;
done &>> $path/Outputs/FeatureCounts/featureCounts.verbose;

Content of featureCounts.R :

In [ ]:
### CODE ###
#!/usr/bin/env Rscript

args = commandArgs(trailingOnly=TRUE)
in_file = paste(args[1], 'bam', sep = '.')
out_file = paste('../FeatureCounts/', args[1], '.tsv', sep = '')

library(Rsubread)

fc = featureCounts(files=in_file,
                   annot.ext="../../Data/Mus_musculus/gencode.vM14.chr_patch_hapl_scaff.annotation.gtf",
                   isGTFAnnotationFile=TRUE,
                   GTF.featureType="exon",
                   GTF.attrType="gene_id",
                   chrAliases="../../Data/Mus_musculus/chr.aliases",
                   allowMultiOverlap=FALSE,
                   ignoreDup=FALSE,
                   isPairedEnd=TRUE,
                   nthreads=8,
                   minMQS=0,
                   maxMOp=10,
                   checkFragLength=FALSE,
                   useMetaFeatures=TRUE,
                   strandSpecific=2,
                   requireBothEndsMapped=TRUE,
                   countChimericFragments=FALSE,
                   primaryOnly=TRUE)
write.table(fc$counts, out_file, append= T, sep='\t', col.names=NA)

#### 4.2.2) Pefanis data

##### a) Paired end

In [ ]:
### CODE ###
path='/run/user/1000/gvfs/smb-share:server=ubox.unilim.fr,share=biscem/BioInformatique/Responsables/Commun/PTCB';
cd $path/Outputs/Star;
for id in 'SRR1302208' 'SRR1302283' 'SRR1302786' 'SRR1302787' 'SRR1302788' 'SRR1302789';
do Rscript $path/Scripts/featureCounts.R $id;
done &>> $path/Outputs/FeatureCounts/featureCounts.verbose;

Content of featureCounts.R :

In [ ]:
### CODE ###
#!/usr/bin/env Rscript

args = commandArgs(trailingOnly=TRUE)
in_file = paste(args[1], 'bam', sep = '.')
out_file = paste('/run/user/1000/gvfs/smb-share:server=ubox.unilim.fr,share=biscem/BioInformatique/Responsables/Commun/PTCB/Outputs/FeatureCounts/', args[1], '.tsv', sep = '')

library(Rsubread)
print(paste('/run/user/1000/gvfs/smb-share:server=ubox.unilim.fr,share=biscem/BioInformatique/Responsables/Commun/PTCB/Star/', in_file, sep = ''))
fc = featureCounts(files=paste('/run/user/1000/gvfs/smb-share:server=ubox.unilim.fr,share=biscem/BioInformatique/Responsables/Commun/PTCB/Star/', in_file, sep = ''),
                   annot.ext="/run/user/1000/gvfs/smb-share:server=ubox.unilim.fr,share=biscem/BioInformatique/Responsables/Commun/PTCB/Data/Mus_musculus/gencode.vM14.chr_patch_hapl_scaff.annotation.gtf",
                   isGTFAnnotationFile=TRUE,
                   GTF.featureType="exon",
                   GTF.attrType="gene_id",
                   chrAliases="/run/user/1000/gvfs/smb-share:server=ubox.unilim.fr,share=biscem/BioInformatique/Responsables/Commun/PTCB/chr.aliases",
                   allowMultiOverlap=FALSE,
                   ignoreDup=FALSE,
                   isPairedEnd=TRUE,
                   nthreads=8,
                   minMQS=0,
                   maxMOp=10,
                   checkFragLength=FALSE,
                   useMetaFeatures=TRUE,
                   strandSpecific=2,
                   requireBothEndsMapped=TRUE,
                   countChimericFragments=FALSE,
                   primaryOnly=TRUE)
write.table(fc$counts, out_file, append= T, sep='\t', col.names=NA)

##### b) Single end

In [ ]:
### CODE ###
path='/run/user/1000/gvfs/smb-share:server=ubox.unilim.fr,share=biscem/BioInformatique/Responsables/Commun/PTCB';
cd $path/Outputs/Star;
for id in 'SRR1302414' 'SRR1302785';
do Rscript $path/Scripts/featureCounts.R $id;
done &>> $path/Outputs/FeatureCounts/featureCounts.verbose;

Content of featureCounts.R :

In [ ]:
### CODE ###
#!/usr/bin/env Rscript

args = commandArgs(trailingOnly=TRUE)
in_file = paste(args[1], 'bam', sep = '.')
out_file = paste('/run/user/1000/gvfs/smb-share:server=ubox.unilim.fr,share=biscem/BioInformatique/Responsables/Commun/PTCB/Outputs/FeatureCounts/', args[1], '.tsv', sep = '')

library(Rsubread)
print(paste('/run/user/1000/gvfs/smb-share:server=ubox.unilim.fr,share=biscem/BioInformatique/Responsables/Commun/PTCB/Star/', in_file, sep = ''))
fc = featureCounts(files=paste('/run/user/1000/gvfs/smb-share:server=ubox.unilim.fr,share=biscem/BioInformatique/Responsables/Commun/PTCB/Star/', in_file, sep = ''),
                   annot.ext="/run/user/1000/gvfs/smb-share:server=ubox.unilim.fr,share=biscem/BioInformatique/Responsables/Commun/PTCB/Data/Mus_musculus/gencode.vM14.chr_patch_hapl_scaff.annotation.gtf",
                   isGTFAnnotationFile=TRUE,
                   GTF.featureType="exon",
                   GTF.attrType="gene_id",
                   chrAliases="/run/user/1000/gvfs/smb-share:server=ubox.unilim.fr,share=biscem/BioInformatique/Responsables/Commun/PTCB/chr.aliases",
                   allowMultiOverlap=FALSE,
                   ignoreDup=FALSE,
                   isPairedEnd=FALSE,
                   nthreads=8,
                   minMQS=0,
                   maxMOp=10,
                   checkFragLength=FALSE,
                   useMetaFeatures=TRUE,
                   strandSpecific=0,
                   countChimericFragments=FALSE,
                   primaryOnly=TRUE)
write.table(fc$counts, out_file, append= T, sep='\t', col.names=NA)

### 4.2) Kallisto

On Genotoul via Kallisto, version 0.43.0 :

#### 4.2.1) CRIBL data

In [ ]:
### CODE ###
#!/bin/bash
#$ -q workq
#$ -m bea
#$ -l mem=8G
#$ -l h_vmem=30G
#

#####Alignment KALLISTO
for id in '3B4_R' '3B4_S' '3pRR_R' '3pRR_S' 'DEL_R' 'DEL_S' 'IRIS_R' 'IRIS_S' 'PAL_R' 'PAL_S' 'WT_R' 'WT_S';
do
kallisto quant -i /home/hbastien/work/PTCB/mm10_transcriptome/Mus_musculus.GRCm38.cdna.all.idx -o kallisto$id'EP' -b 100 --rf-stranded /home/escaon/save/PTCB/Reads_raw_Nice_trim/$id'_1_EP_paired.fastq.gz' /home/escaon/save/PTCB/Reads_raw_Nice_trim/$id'_2_EP_paired.fastq.gz'
done
for id in '3pRR_R' '3pRR_S' 'EU_R' 'EU_S' 'MAR_R' 'MAR_S' 'WT_R' 'WT_S';
do
kallisto quant -i /home/hbastien/work/PTCB/mm10_transcriptome/Mus_musculus.GRCm38.cdna.all.idx -o kallisto$id'YD' -b 100 --rf-stranded /home/escaon/save/PTCB/Reads_raw_Nice_trim/$id'_1_YD_paired.fastq.gz' /home/escaon/save/PTCB/Reads_raw_Nice_trim/$id'_2_YD_paired.fastq.gz'
done

In [ ]:
### CODE ###
qsub manageKallisto.sh

#### 4.2.2) Pefanis data

##### a) Paired end

In [ ]:
### CODE ###
#!/bin/bash
#$ -q workq
#$ -m bea
#$ -l mem=8G
#$ -l h_vmem=30G
#

#####Alignment KALLISTO PE
for id in 'SRR1302208' 'SRR1302283' 'SRR1302786' 'SRR1302787' 'SRR1302788' 'SRR1302789';
do
kallisto quant -i /home/hbastien/work/PTCB/mm10_transcriptome/Mus_musculus.GRCm38.cdna.all.idx -o kallisto$id -b 100 /home/hbastien/work/PTCB/Pefanis_rebuild/'Trimmed_paired_'$id'_1.fastq' /home/hbastien/work/PTCB/Pefanis_rebuild/'Trimmed_paired_'$id'_2.fastq'
done

In [ ]:
### CODE ###
qsub manageKallistoPE.sh

##### b) Single end

In [ ]:
### CODE ###
#!/bin/bash
#$ -q workq
#$ -m bea
#$ -l mem=8G
#$ -l h_vmem=30G
#

#####Alignment KALLISTO SE
for id in 'SRR1302414' 'SRR1302785';
do
kallisto quant -i /home/hbastien/work/PTCB/mm10_transcriptome/Mus_musculus.GRCm38.cdna.all.idx -o kallisto$id -b 100 --single -l 100 -s 20 /home/hbastien/work/PTCB/Pefanis_rebuild/'Trimmed_'$id'_1.fastq'
done

In [ ]:
### CODE ###
qsub manageKallistoSE.sh

##### Retrieve count files locally :

In [ ]:
### CODE ###
sudo scp -r hbastien@genotoul.toulouse.inra.fr:/home/hbastien/work/PTCB/kallisto* /run/user/1000/gvfs/smb-share:server=ubox.unilim.fr,share=biscem/BioInformatique/Responsables/Commun/PTCB/Outputs/Kallisto

### 4.3) Downstream analysis compatibility
Slightly modify featureCounts output to make it compatible with future steps of the analysis.
We want our gene names to be the same than those contained in ensembl when retrieved with biomart.

#### 4.3.1) FeatureCounts

Remove spreadsheet hidden ' " ' for downstream compatility with our awk manipulation :

In [ ]:
### CODE ###
cd ~/Desktop/PTCB/BISCEm/Outputs/FeatureCounts;
for f in *.tsv;
do sed 's/\"//g' $f > ${f%.*}".temp.tsv";
done;

Use awk sub to convert "ENSMUSG00000098104.12" -> "ENSMUSG00000098104" :

In [ ]:
### CODE ###
for f in *.temp.tsv;
do awk -F '\t' -v OFS='\t' 'NR > 1 {sub(/\.[0-9]*/, "", $1)} 1' $f > ${f%%.*}"_ensembl_gene_id.tsv";
done;
rm *.temp.tsv;

Check that the removal of ".[0-9]?" from the gene id column did not change the number of unique gene ids :

In [ ]:
### CODE ###
for f in *.tsv;
do awk -F '\t' '{print $1}' $f | sort -u | wc -l;
done;

We did show that transforming gene name "ENSMUSG00000098104.12" -> "ENSMUSG00000098104" did not change the number unique gene names.

#### 4.3.2) Kallisto

Remove spreadsheet hidden ' " ' for downstream compatility with our awk manipulation :

In [ ]:
### CODE ###
cd /run/user/1000/gvfs/smb-share:server=ubox.unilim.fr,share=biscem/BioInformatique/Responsables/Commun/PTCB/Outputs/Kallisto;
for d in ./*;
do sed 's/\"//g' $d/"abundance.tsv" > $d/"abundance.temp.tsv";
done;

Use awk sub to convert "ENSMUSG00000098104.12" -> "ENSMUSG00000098104" :

In [ ]:
### CODE ###
for d in ./*;
do awk -F '\t' -v OFS='\t' 'NR > 1 {sub(/\.[0-9]*/, "", $1)} 1' $d/"abundance.tsv" > $d/"abundance_ensembl_gene_id.tsv";
rm $d/*.temp.tsv;
done;

Check that the removal of ".[0-9]?" from the gene id column did not change the number of unique gene ids :

In [ ]:
### CODE ###
for d in ./*;
do awk -F '\t' '{print $1}' $d/"abundance.tsv" | sort -u | wc -l;
done;

We did show that transforming gene name "ENSMUSG00000098104.12" -> "ENSMUSG00000098104" did not change the number unique gene names.

## 5) DE analysis

Perfomed with DESeq2_1.16.1 (using Bioconductor 3.5 (BiocInstaller 1.26.0), R 3.4.1 (2017-06-30)) :

##### A quick word regarding comparisions with no replicates

From DESeq2 documentation (https://bioconductor.org/packages/release/bioc/vignettes/DESeq2/inst/doc/DESeq2.html#can-i-use-deseq2-to-analyze-a-dataset-without-replicates) : <br>
Without replicates, a warning is printed, that the samples are treated as replicates for estimation of dispersion. This kind of analysis is only useful for exploring the data, but will not provide the kind of proper statistical inference on differences between groups. Without biological replicates, it is not possible to estimate the biological variability of each gene.

### 5.1) Resting vs Stimulated for the same mouse model + MA and Volcano plots

#### 5.1.1) No replicates

##### With FeatureCounts :

In [1]:
### CODE ###
pathway='/run/user/1000/gvfs/smb-share:server=ubox.unilim.fr,share=biscem/BioInformatique/Responsables/Commun/PTCB';
cd $pathway;
arrayR=('3B4_R_EP' '3pRR_R_EP' 'DEL_R_EP' 'IRIS_R_EP' 'PAL_R_EP' 'WT_R_EP' '3pRR_R_YD' 'EU_R_YD' 'MAR_R_YD' 'WT_R_YD')
arrayS=('3B4_S_EP' '3pRR_S_EP' 'DEL_S_EP' 'IRIS_S_EP' 'PAL_S_EP' 'WT_S_EP' '3pRR_S_YD' 'EU_S_YD' 'MAR_S_YD' 'WT_S_YD')

length=${#arrayR[@]}
for ((i=0;i<=$length-1;i++));
do
Rscript $pathway/Scripts/deseq2_R_vs_S_no_replicates_fc.R ${arrayR[$i]} ${arrayS[$i]} &>> $pathway/Outputs/DESeq2/Verbose/deseq2_R_vs_S_no_replicates_fc.verbose;
done;

Content of deseq2_R_vs_S_no_replicates_fc.R :

In [1]:
### CODE ###

##### With Kallisto :

In [2]:
### CODE ###
pathway='/run/user/1000/gvfs/smb-share:server=ubox.unilim.fr,share=biscem/BioInformatique/Responsables/Commun/PTCB';
cd $pathway;
arrayR=('3B4_R_EP' '3pRR_R_EP' 'DEL_R_EP' 'IRIS_R_EP' 'PAL_R_EP' 'WT_R_EP' '3pRR_R_YD' 'EU_R_YD' 'MAR_R_YD' 'WT_R_YD')
arrayS=('3B4_S_EP' '3pRR_S_EP' 'DEL_S_EP' 'IRIS_S_EP' 'PAL_S_EP' 'WT_S_EP' '3pRR_S_YD' 'EU_S_YD' 'MAR_S_YD' 'WT_S_YD')

length=${#arrayR[@]}
for ((i=0;i<=$length-1;i++));
do
Rscript $pathway/Scripts/deseq2_R_vs_S_no_replicates_ka.R ${arrayR[$i]} ${arrayS[$i]} &>> $pathway/Outputs/DESeq2/Verbose/deseq2_R_vs_S_no_replicates_ka.verbose;
done;

Content of deseq2_R_vs_S_no_replicates_ka.R :

In [ ]:
### CODE ###

#### 5.1.2) With replicates when available (i.e., 1 sample coming from Eric Pinaud and 1 from Yves Denizot)

##### With FeatureCounts :

In [3]:
### CODE ###
pathway='/run/user/1000/gvfs/smb-share:server=ubox.unilim.fr,share=biscem/BioInformatique/Responsables/Commun/PTCB';
cd $pathway;
arrayR1=('3pRR_R_EP' 'WT_R_EP')
arrayR2=('3pRR_R_YD' 'WT_R_YD')
arrayS1=('3pRR_S_EP' 'WT_S_EP')
arrayS2=('3pRR_S_YD' 'WT_S_YD')

length=${#arrayR1[@]}
for ((i=0;i<=$length-1;i++));
do
Rscript $pathway/Scripts/deseq2_R_vs_S_with_replicates_fc.R ${arrayR1[$i]} ${arrayR2[$i]} ${arrayS1[$i]} ${arrayS2[$i]} &>> $pathway/Outputs/DESeq2/Verbose/deseq2_R_vs_S_with_replicates_fc.verbose;
done;

Content of deseq2_R_vs_S_with_replicates_fc.R :

In [ ]:
### CODE ###

##### With Kallisto :

In [4]:
### CODE ###
pathway='/run/user/1000/gvfs/smb-share:server=ubox.unilim.fr,share=biscem/BioInformatique/Responsables/Commun/PTCB';
cd $pathway;
arrayR1=('3pRR_R_EP' 'WT_R_EP')
arrayR2=('3pRR_R_YD' 'WT_R_YD')
arrayS1=('3pRR_S_EP' 'WT_S_EP')
arrayS2=('3pRR_S_YD' 'WT_S_YD')

length=${#arrayR1[@]}
for ((i=0;i<=$length-1;i++));
do
Rscript $pathway/Scripts/deseq2_R_vs_S_with_replicates_ka.R ${arrayR1[$i]} ${arrayR2[$i]} ${arrayS1[$i]} ${arrayS2[$i]} &>> $pathway/Outputs/DESeq2/Verbose/deseq2_R_vs_S_with_replicates_ka.verbose;
done;

Content of deseq2_R_vs_S_with_replicates_ka.R :

In [ ]:
### CODE ###

#### 5.1.3) Pooled R vs S with all mouse models & WT (EP, YD, EP+YD)

Dunno if this comparison is useful

In [ ]:
### CODE ###
path='/home/erwann/Desktop/PTCB/BISCEm';
Rscript $path/Scripts/deseq2_R_vs_S_pooled.R &>> $path/Outputs/DESeq2/Verbose/deseq2_R_vs_S_pooled.verbose;

Content of deseq2_R_vs_S_pooled.R for the first comparison :

In [ ]:
### CODE ###

### 5.2) Mutant vs WT (for R and for S)

#### 5.2.1) No replicates

##### With FeatureCounts :

In [5]:
### CODE ###
pathway='/run/user/1000/gvfs/smb-share:server=ubox.unilim.fr,share=biscem/BioInformatique/Responsables/Commun/PTCB';
cd $pathway;
array=('3B4_R_EP' '3B4_S_EP' '3pRR_R_EP' '3pRR_S_EP' 'DEL_R_EP' 'DEL_S_EP' 'IRIS_R_EP' 'IRIS_S_EP' 'PAL_R_EP' 'PAL_S_EP' '3pRR_R_YD' '3pRR_S_YD' 'EU_R_YD' 'EU_S_YD' 'MAR_R_YD' 'MAR_S_YD')

length=${#array[@]}
for ((i=0;i<=$length-1;i++));
do
Rscript $pathway/Scripts/deseq2_mutant_vs_wt_no_replicates_fc.R ${array[$i]} &>> $pathway/Outputs/DESeq2/Verbose/deseq2_mutant_vs_wt_no_replicates_fc.verbose;
done;

Content of deseq2_mutant_vs_wt_no_replicates_fc.R :

In [2]:
### CODE ###

##### With Kallisto :

In [6]:
### CODE ###
pathway='/run/user/1000/gvfs/smb-share:server=ubox.unilim.fr,share=biscem/BioInformatique/Responsables/Commun/PTCB';
cd $pathway;
array=('3B4_R_EP' '3B4_S_EP' '3pRR_R_EP' '3pRR_S_EP' 'DEL_R_EP' 'DEL_S_EP' 'IRIS_R_EP' 'IRIS_S_EP' 'PAL_R_EP' 'PAL_S_EP' '3pRR_R_YD' '3pRR_S_YD' 'EU_R_YD' 'EU_S_YD' 'MAR_R_YD' 'MAR_S_YD')

length=${#array[@]}
for ((i=0;i<=$length-1;i++));
do
Rscript $pathway/Scripts/deseq2_mutant_vs_wt_no_replicates_ka.R ${array[$i]} &>> $pathway/Outputs/DESeq2/Verbose/deseq2_mutant_vs_wt_no_replicates_ka.verbose;
done;

Content of deseq2_mutant_vs_wt_no_replicates_ka.R :

In [ ]:
### CODE ###

#### 5.2.2) With replicates when available (i.e., 1 sample coming from Eric Pinaud "EP" and 1 from Yves Denizot "YD")

##### 5.2.2.1) CRIBL

##### With FeatureCounts :

In [7]:
### CODE ###
pathway='/run/user/1000/gvfs/smb-share:server=ubox.unilim.fr,share=biscem/BioInformatique/Responsables/Commun/PTCB';
cd $pathway;
arrayR1=('3pRR_R_EP' '3pRR_S_EP')
arrayR2=('3pRR_R_YD' '3pRR_S_YD')
length=${#arrayR1[@]}
for ((i=0;i<=$length-1;i++));
do
Rscript $pathway/Scripts/deseq2_mutant_vs_wt_with_replicates_fc.R ${arrayR1[$i]} ${arrayR2[$i]} &>> $pathway/Outputs/DESeq2/Verbose/deseq2_mutant_vs_wt_with_replicates_fc.verbose;
done;

Content of deseq2_mutant_vs_wt_with_replicates_fc.R :

In [ ]:
### CODE ###

##### With Kallisto :

In [8]:
### CODE ###
pathway='/run/user/1000/gvfs/smb-share:server=ubox.unilim.fr,share=biscem/BioInformatique/Responsables/Commun/PTCB';
cd $pathway;
arrayR1=('3pRR_R_EP' '3pRR_S_EP')
arrayR2=('3pRR_R_YD' '3pRR_S_YD')
length=${#arrayR1[@]}
for ((i=0;i<=$length-1;i++));
do
Rscript $pathway/Scripts/deseq2_mutant_vs_wt_with_replicates_ka.R ${arrayR1[$i]} ${arrayR2[$i]} &>> $pathway/Outputs/DESeq2/Verbose/deseq2_mutant_vs_wt_with_replicates_ka.verbose;
done;

Content of deseq2_mutant_vs_wt_with_replicates_ka.R :

In [ ]:
### CODE ###

##### 5.2.2.2) Pefanis

##### With FeatureCounts :

In [23]:
### CODE ###
pathway='/run/user/1000/gvfs/smb-share:server=ubox.unilim.fr,share=biscem/BioInformatique/Responsables/Commun/PTCB';
cd $pathway;
Rscript $pathway/Scripts/deseq2_mutant_vs_wt_pefanis_with_replicates_fc.R SRR1302208 SRR1302283 SRR1302414 SRR1302785 SRR1302786 SRR1302787 SRR1302788 SRR1302789 &>> $pathway/Outputs/DESeq2/Verbose/deseq2_mutant_vs_wt_pefanis_with_replicates_fc.verbose;

Content of deseq2_mutant_vs_wt_pefanis_with_replicates_fc.R :

In [ ]:
### CODE ###

##### With Kallisto :

In [10]:
### CODE ###
pathway='/run/user/1000/gvfs/smb-share:server=ubox.unilim.fr,share=biscem/BioInformatique/Responsables/Commun/PTCB';
cd $pathway;
Rscript $pathway/Scripts/deseq2_mutant_vs_wt_pefanis_with_replicates_ka.R SRR1302208 SRR1302283 SRR1302414 SRR1302785 SRR1302786 SRR1302787 SRR1302788 SRR1302789 &>> $pathway/Outputs/DESeq2/Verbose/deseq2_mutant_vs_wt_pefanis_with_replicates_ka.verbose;

Content of deseq2_mutant_vs_wt_pefanis_with_replicates_ka.R :

In [ ]:
### CODE ###

### 5.3) Output formatting

##### Point to Comma :

In [ ]:
### CODE ###
cd ~/Desktop/PTCB/BISCEm/Outputs/DESeq2/Mutant_vs_WT_no_replicates;
mkdir -p Comma;
for f in *.tsv;
do sed 's/\./,/g' $f > Comma/${f%.*}'_comma.tsv';
done;

cd ~/Desktop/PTCB/BISCEm/Outputs/DESeq2/Mutant_vs_WT_with_replicates;
mkdir -p Comma;
for f in *.tsv;
do sed 's/\./,/g' $f > Comma/${f%.*}'_comma.tsv';
done;

cd ~/Desktop/PTCB/BISCEm/Outputs/DESeq2/R_vs_S_no_replicates;
mkdir -p Comma;
for f in *.tsv;
do sed 's/\./,/g' $f > Comma/${f%.*}'_comma.tsv';
done;

cd ~/Desktop/PTCB/BISCEm/Outputs/DESeq2/R_vs_S_pooled;
mkdir -p Comma;
for f in *.tsv;
do sed 's/\./,/g' $f > Comma/${f%.*}'_comma.tsv';
done;

cd ~/Desktop/PTCB/BISCEm/Outputs/DESeq2/R_vs_S_with_replicates;
mkdir -p Comma;
for f in *.tsv;
do sed 's/\./,/g' $f > Comma/${f%.*}'_comma.tsv';
done;

##### Top DE genes (no-replicates) :

In [ ]:
### CODE ###
cd ~/Desktop/PTCB/BISCEm/Outputs/DESeq2/Mutant_vs_WT_no_replicates;
mkdir -p Filtered_top_DE;
for f in *.tsv;
do awk -F '\t' \
       'NR<2 \
       {print $0}' \
       $f \
       > Filtered_top_DE/${f%.*}"_top_DE.tsv";
   awk -F '\t' \
       'NR>1 \
       {if($5 > 2 || $5 < -2) \
        if($8 < 0.05) \
        if($10 >= 100 || $11 >= 100) \
        print $0}' \
        $f \
        >> Filtered_top_DE/${f%.*}"_top_DE.tsv";
done;

cd ~/Desktop/PTCB/BISCEm/Outputs/DESeq2/R_vs_S_no_replicates;
mkdir -p Filtered_top_DE;
for f in *.tsv;
do awk -F '\t' \
       'NR<2 \
       {print $0}' \
       $f \
       > Filtered_top_DE/${f%.*}"_top_DE.tsv";
   awk -F '\t' \
       'NR>1 \
       {if($5 > 2 || $5 < -2) \
        if($8 < 0.05) \
        if($10 >= 100 || $11 >= 100) \
        print $0}' \
        $f \
        >> Filtered_top_DE/${f%.*}"_top_DE.tsv";
done;

##### Top DE genes (with-replicates) :

In [ ]:
### CODE ###
cd ~/Desktop/PTCB/BISCEm/Outputs/DESeq2/Mutant_vs_WT_with_replicates;
mkdir -p Filtered_top_DE;
for f in *.tsv;
do awk -F '\t' \
       'NR<2 \
       {print $0}' \
       $f \
       > Filtered_top_DE/${f%.*}"_top_DE.tsv";
   awk -F '\t' \
       'NR>1 \
       {if($9 < 0.05) \
        print $0}' \
        $f \
        >> Filtered_top_DE/${f%.*}"_top_DE.tsv";
done;

cd ~/Desktop/PTCB/BISCEm/Outputs/DESeq2/R_vs_S_pooled;
mkdir -p Filtered_top_DE;
for f in *.tsv;
do awk -F '\t' \
       'NR<2 \
       {print $0}' \
       $f \
       > Filtered_top_DE/${f%.*}"_top_DE.tsv";
   awk -F '\t' \
       'NR>1 \
       {if($9 < 0.05) \
        print $0}' \
        $f \
        >> Filtered_top_DE/${f%.*}"_top_DE.tsv";
done;

cd ~/Desktop/PTCB/BISCEm/Outputs/DESeq2/R_vs_S_with_replicates;
mkdir -p Filtered_top_DE;
for f in *.tsv;
do awk -F '\t' \
       'NR<2 \
       {print $0}' \
       $f \
       > Filtered_top_DE/${f%.*}"_top_DE.tsv";
   awk -F '\t' \
       'NR>1 \
       {if($9 < 0.05) \
        print $0}' \
        $f \
        >> Filtered_top_DE/${f%.*}"_top_DE.tsv";
done;

##### Genes of interest list for MAR vs WT :

List "genes_of_interest_mar.tsv" is constructed from :
- All members of the ARID family
- All members of thee SATB family
- Cux1
- AID off targets genes in Liu et al., Nature Letters, 02/2008, Supp Fig1
- Genes on the first page of Yamane et al., Nature Immunology, 01/2011, S1 Table

In [ ]:
### CODE ###
# On filtered results
cd ~/Desktop/PTCB/BISCEm/Outputs/DESeq2/Mutant_vs_WT_no_replicates/Filtered_top_DE;
while read line;
do grep $line MAR_vs_WT_R_YD_top_DE.tsv;
done < ~/Desktop/PTCB/BISCEm/Mar/genes_of_interest_mar.tsv;
# Nothing
while read line;
do grep $line MAR_vs_WT_S_YD_top_DE.tsv;
done < ~/Desktop/PTCB/BISCEm/Mar/genes_of_interest_mar.tsv;
# Nothing

# Given that there was nothing on filtered, at least output those in raw data
cd ~/Desktop/PTCB/BISCEm/Outputs/DESeq2/Mutant_vs_WT_no_replicates;
while read line;
do grep $line MAR_vs_WT_R_YD.tsv;
done < ~/Desktop/PTCB/BISCEm/Mar/genes_of_interest_mar.tsv > ~/Desktop/PTCB/BISCEm/Mar/MAR_vs_WT_R_YD_gon.tsv;
while read line;
do grep $line MAR_vs_WT_S_YD.tsv;
done < ~/Desktop/PTCB/BISCEm/Mar/genes_of_interest_mar.tsv > ~/Desktop/PTCB/BISCEm/Mar/MAR_vs_WT_S_YD_gon.tsv;

### 5.5) Heatmap

#### 5.5.1) Heatmap R vs S

In [28]:
### CODE ###
pathway='/run/user/1000/gvfs/smb-share:server=ubox.unilim.fr,share=biscem/BioInformatique/Responsables/Commun/PTCB';
cd $pathway;
array=('3B4_EP' '3pRR_EP' '3pRR_YD' 'DEL_EP' 'IRIS_EP' 'PAL_EP' 'EU_YD' 'MAR_YD' 'WT_EP' 'WT_YD')

length=${#array[@]}
for ((i=0;i<=$length-1;i++));
do
Rscript $pathway/Scripts/heatmap_RvS.R ${array[$i]} &>> $pathway/Outputs/Heatmap/Verbose/heatmap_RvS.verbose;
done;

Content of deseq2_heatmap_RvS_fc_nocluster.R :

In [ ]:
### CODE ###

#### 5.5.2) Heatmap Mutant vs WT

##### a) Resting heatmap

In [27]:
### CODE ###
pathway='/run/user/1000/gvfs/smb-share:server=ubox.unilim.fr,share=biscem/BioInformatique/Responsables/Commun/PTCB';
cd $pathway;
array=('3B4_vs_WT_R_EP' '3pRR_vs_WT_R_EP' '3pRR_vs_WT_R_YD' 'DEL_vs_WT_R_EP' 'IRIS_vs_WT_R_EP' 'PAL_vs_WT_R_EP' 'EU_vs_WT_R_YD' 'MAR_vs_WT_R_YD')

length=${#array[@]}
for ((i=0;i<=$length-1;i++));
do
Rscript $pathway/Scripts/heatmap_MutantvWT_R.R ${array[$i]} &>> $pathway/Outputs/Heatmap/Verbose/heatmap_MutantvWT_R.verbose;
done;

Content of heatmap_MutantvWT_R.R :

In [3]:
### CODE ###

##### b) Stimulated heatmap

##### CRIBL

In [26]:
### CODE ###
pathway='/run/user/1000/gvfs/smb-share:server=ubox.unilim.fr,share=biscem/BioInformatique/Responsables/Commun/PTCB';
cd $pathway;
array=('3B4_vs_WT_S_EP' '3pRR_vs_WT_S_EP' '3pRR_vs_WT_S_YD' 'DEL_vs_WT_S_EP' 'IRIS_vs_WT_S_EP' 'PAL_vs_WT_S_EP' 'EU_vs_WT_S_YD' 'MAR_vs_WT_S_YD')

length=${#array[@]}
for ((i=0;i<=$length-1;i++));
do
Rscript $pathway/Scripts/heatmap_MutantvWT_S.R ${array[$i]} &>> $pathway/Outputs/Heatmap/Verbose/heatmap_MutantvWT_S.verbose;
done;

Content of heatmap_MutantvWT_S.R :

In [8]:
### CODE ###

##### Pefanis

In [25]:
### CODE ###
pathway='/run/user/1000/gvfs/smb-share:server=ubox.unilim.fr,share=biscem/BioInformatique/Responsables/Commun/PTCB';
cd $pathway;
array=('KO_vs_WT_pefanis_dup')

length=${#array[@]}
for ((i=0;i<=$length-1;i++));
do
Rscript $pathway/Scripts/heatmap_MutantvWT_S_pefanis.R ${array[$i]} &>> $pathway/Outputs/Heatmap/Verbose/heatmap_MutantvWT_S_pefanis.verbose;
done;

Content of heatmap_MutantvWT_S_pefanis.R

In [ ]:
### CODE ###

#### 5.5.3) Normalized counts heatmap

The goal is to plot all significantly DE genes on a heatmap for all DESeq2 comparisons, even 1vs1 comparisons. The log2 of normalized counts is best suited for this purpose, as explained here : https://support.bioconductor.org/p/76712/. <br>

For instance, consider 3B4_R vs 3B4_S comparison. To obtain a heatmap showing DE genes, we need a variable with values for both R and S. Log2(fold_change) cannot achieve this. Counts can fulfil this task, but we want to normalize them to avoid misinterpretation. It is suggested to use variance stabilized normalized counts (varianceStabilizingTransformation function). Given that the DE genes we consider significant are already filtered (on log2(fold_change), p_value and counts), they have robust counts and in those situations, varianceStabilizingTransformation is equivalent to using log2(normalized_count). Thus for the sake of simplicity, we will use log2(normalized_count).

### 5.6) Normalized visualisation

After DE genes have been called, biologists want to go back to "the gene level" and visualize said DE genes in a genome browser, with associated read pileup / coverage for each condition. <br>

Two things have to be considered here : <br>
- Direct visualization from raw BAMs is skewed, because DESeq2 called the DE genes based on normalized counts, not raw counts. Thus we need to "normalize the BAM". This is achieved by retrieving for each DESeq2 comparision the result of the "sizeFactors(dds)" command.  <br>
See p5 of https://bioconductor.org/packages/release/bioc/manuals/DESeq2/man/DESeq2.pdf : "whether or not to divide the counts by the size factors".  <br>
An example : Consider the 1vs1 comparison of WT_R_EP vs WT_S_EP. Associated size factors are 0.8560343 (R) and 1.1681774 (S). In the DESeq2 output, take a random result : "ENSMUSG00000000078", which have associated normalized counts : 10241.41 (R) & 925.37 (S). Now check that there were indeed obtained by dividing raw featureCounts counts by size factor : Count value for R : 8767, divided by 0.8560343 = 10241.41 & count value for S : 1081, divided by 1.1681774 = 925.37. <br>


- Our data is paired and strand-specific, thus we can and should split BAM for the read pairs on the forward and on the reverse strand. This is achieved with the "filterRNAstrand" option within the bamCoverage software.

In [ ]:
### CODE ###
# Set paths & folders for normalized BAMs creation
biscem='/home/erwann/Desktop/PTCB/BISCEm';
mkdir -p $biscem/Outputs/Normalized_BAM/R_vs_S_no_replicates;
mkdir -p $biscem/Outputs/Normalized_BAM/R_vs_S_with_replicates;
mkdir -p $biscem/Outputs/Normalized_BAM/Mutant_vs_WT_no_replicates;
mkdir -p $biscem/Outputs/Normalized_BAM/Mutant_vs_WT_with_replicates;
mkdir -p $biscem/Outputs/Normalized_BAM/Verbose;

############################
### R_vs_S_no_replicates ###
############################
# Set scales for R_vs_S_no_replicates
scale_3B4_R_EP=1.0019940;
scale_3B4_S_EP=0.9980099;
scale_3pRR_R_EP=1.0258757;
scale_3pRR_S_EP=0.9747769;
scale_DEL_R_EP=0.9876488;
scale_DEL_S_EP=1.0125057;
scale_IRIS_R_EP=1;
scale_IRIS_S_EP=1;
scale_PAL_R_EP=0.9251799;
scale_PAL_S_EP=1.0808709;
scale_WT_R_EP=0.8560343;
scale_WT_S_EP=1.1681774;
scale_3pRR_R_YD=1.4699883;
scale_3pRR_S_YD=0.6802775;
scale_EU_R_YD=1.0939116;
scale_EU_S_YD=0.9141506;
scale_MAR_R_YD=1.7791104;
scale_MAR_S_YD=0.5620787;
scale_WT_R_YD=0.9906975;
scale_WT_S_YD=1.0093899;

# Proceed BAMs with bamCoverage
cd $biscem/Outputs/Mapped_vs_GRCm38.p5;
for f in *.bam;
do scale=$(eval printf "$""scale_"${f%.*});
   bamCoverage --bam $f \
               --outFileName $biscem/Outputs/Normalized_BAM/R_vs_S_no_replicates/${f%.*}"_forward_strand.bedgraph" \
               --outFileFormat bedgraph \
               --scaleFactor $scale \
               --filterRNAstrand forward \
               --binSize 1 \
               --numberOfProcessors max \
               --verbose;
   bamCoverage --bam $f \
               --outFileName $biscem/Outputs/Normalized_BAM/R_vs_S_no_replicates/${f%.*}"_reverse_strand.bedgraph" \
               --outFileFormat bedgraph \
               --scaleFactor $scale \
               --filterRNAstrand reverse \
               --binSize 1 \
               --numberOfProcessors max \
               --verbose;
done &>> $biscem/Outputs/Normalized_BAM/Verbose/r_vs_s_no_replicates.verbose;

##############################
### R_vs_S_with_replicates ###
##############################
# Set scales for R_vs_S_with_replicates
scale_3pRR_R_EP=1.5761964;
scale_3pRR_R_YD=0.9873693;
scale_3pRR_S_EP=1.4713837;
scale_3pRR_S_YD=0.4476779;
scale_WT_R_EP=0.9898616;
scale_WT_R_YD=0.8120548;
scale_WT_S_EP=1.4213428;
scale_WT_S_YD=0.8887980;

# Proceed BAMs with bamCoverage
cd $biscem/Outputs/Mapped_vs_GRCm38.p5;
for id in '3pRR_R_EP' '3pRR_R_YD' '3pRR_S_EP' '3pRR_S_YD' 'WT_R_EP' 'WT_R_YD' 'WT_S_EP' 'WT_S_YD';
do scale=$(eval printf "$""scale_"$id);
   bamCoverage --bam $id.bam \
               --outFileName $biscem/Outputs/Normalized_BAM/R_vs_S_with_replicates/$id"_forward_strand.bedgraph" \
               --outFileFormat bedgraph \
               --scaleFactor $scale \
               --filterRNAstrand forward \
               --binSize 1 \
               --numberOfProcessors max \
               --verbose;
   bamCoverage --bam $id.bam \
               --outFileName $biscem/Outputs/Normalized_BAM/R_vs_S_with_replicates/$id"_reverse_strand.bedgraph" \
               --outFileFormat bedgraph \
               --scaleFactor $scale \
               --filterRNAstrand reverse \
               --binSize 1 \
               --numberOfProcessors max \
               --verbose;
done &>> $biscem/Outputs/Normalized_BAM/Verbose/r_vs_s_with_replicates.verbose;

##################################
### Mutant_vs_WT_no_replicates ###
##################################
# Set scales for Mutant_vs_WT_no_replicates
scale_WT_3B4_R_EP=0.8551271; 
scale_3B4_R_EP=1.1694168;
scale_WT_3B4_S_EP=1.0536938;
scale_3B4_S_EP=0.9490423;
scale_WT_3pRR_R_EP=0.8129696;
scale_3pRR_R_EP=1.2300583;
scale_WT_3pRR_S_EP=1.0164029;
scale_3pRR_S_EP=0.9838618;
scale_WT_DEL_R_EP=0.8951107;
scale_DEL_R_EP=1.1171802;
scale_WT_DEL_S_EP=1.0173134;
scale_DEL_S_EP=0.9829813;
scale_WT_IRIS_R_EP=0.8257228;
scale_IRIS_R_EP=1.2110601;
scale_WT_IRIS_S_EP=0.9916549;
scale_IRIS_S_EP=1.0084153;
scale_WT_PAL_R_EP=0.8831167;
scale_PAL_R_EP=1.1323531;
scale_WT_PAL_S_EP=0.9969528;
scale_PAL_S_EP=1.0030566;
scale_WT_3pRR_R_YD=0.9364606;
scale_3pRR_R_YD=1.0678505;
scale_WT_3pRR_S_YD=1.4142136;
scale_3pRR_S_YD=0.7071068;
scale_WT_EU_R_YD=1.1547005;
scale_EU_R_YD=0.8660254;
scale_WT_EU_S_YD=1.3740364;
scale_EU_S_YD=0.7277828;
scale_WT_MAR_R_YD=0.8312094;
scale_MAR_R_YD=1.2030663;
scale_WT_MAR_S_YD=1.544631;
scale_MAR_S_YD=0.647404;

# Proceed BAMs with bamCoverage
cd $biscem/Outputs/Mapped_vs_GRCm38.p5;
for id in '3B4_R_EP' '3B4_S_EP' '3pRR_R_EP' '3pRR_S_EP' 'DEL_R_EP' 'DEL_S_EP' 'IRIS_R_EP' 'IRIS_S_EP' 'PAL_R_EP' 'PAL_S_EP' '3pRR_R_YD' '3pRR_S_YD' 'EU_R_YD' 'EU_S_YD' 'MAR_R_YD' 'MAR_S_YD';
do scale=$(eval printf "$""scale_"$id);
   bamCoverage --bam $id.bam \
               --outFileName $biscem/Outputs/Normalized_BAM/Mutant_vs_WT_no_replicates/$id"_forward_strand.bedgraph" \
               --outFileFormat bedgraph \
               --scaleFactor $scale \
               --filterRNAstrand forward \
               --binSize 1 \
               --numberOfProcessors max \
               --verbose;
   bamCoverage --bam $id.bam \
               --outFileName $biscem/Outputs/Normalized_BAM/Mutant_vs_WT_no_replicates/$id"_reverse_strand.bedgraph" \
               --outFileFormat bedgraph \
               --scaleFactor $scale \
               --filterRNAstrand reverse \
               --binSize 1 \
               --numberOfProcessors max \
               --verbose;
   scale=$(eval printf "$""scale_WT_"$id);
   mutant=${id%%_*};
   wt_file=$(echo $id | sed 's/'$mutant'/WT/g');
   bamCoverage --bam $wt_file.bam \
               --outFileName $biscem/Outputs/Normalized_BAM/Mutant_vs_WT_no_replicates/"WT_"$id"_forward_strand.bedgraph" \
               --outFileFormat bedgraph \
               --scaleFactor $scale \
               --filterRNAstrand forward \
               --binSize 1 \
               --numberOfProcessors max \
               --verbose;
   bamCoverage --bam $wt_file.bam \
               --outFileName $biscem/Outputs/Normalized_BAM/Mutant_vs_WT_no_replicates/"WT_"$id"_reverse_strand.bedgraph" \
               --outFileFormat bedgraph \
               --scaleFactor $scale \
               --filterRNAstrand reverse \
               --binSize 1 \
               --numberOfProcessors max \
               --verbose;
done &>> $biscem/Outputs/Normalized_BAM/Verbose/mutant_vs_wt_no_replicates.verbose;

####################################
### Mutant_vs_WT_with_replicates ###
####################################
# Set scales for Mutant_vs_WT_with_replicates
scale_WT_R_EP=0.9528320;
scale_WT_R_YD=0.7785160;
scale_threepRR_R_EP=1.5078916;
scale_threepRR_R_YD=0.8942223;
scale_WT_S_EP=1.5711080;
scale_WT_S_YD=0.9494035;
scale_threepRR_S_EP=1.5365522;
scale_threepRR_S_YD=0.4591498;

# Proceed BAMs with bamCoverage
cd $biscem/Outputs/Mapped_vs_GRCm38.p5;
for id in 'WT_R_EP' 'WT_R_YD' 'threepRR_R_EP' 'threepRR_R_YD' 'WT_S_EP' 'WT_S_YD' 'threepRR_S_EP' 'threepRR_S_YD';
do scale=$(eval printf "$""scale_"$id);
   bamCoverage --bam $id.bam \
               --outFileName $biscem/Outputs/Normalized_BAM/Mutant_vs_WT_with_replicates/$id"_forward_strand.bedgraph" \
               --outFileFormat bedgraph \
               --scaleFactor $scale \
               --filterRNAstrand forward \
               --binSize 1 \
               --numberOfProcessors max \
               --verbose;
   bamCoverage --bam $id.bam \
               --outFileName $biscem/Outputs/Normalized_BAM/Mutant_vs_WT_with_replicates/$id"_reverse_strand.bedgraph" \
               --outFileFormat bedgraph \
               --scaleFactor $scale \
               --filterRNAstrand reverse \
               --binSize 1 \
               --numberOfProcessors max \
               --verbose;
done &>> $biscem/Outputs/Normalized_BAM/Verbose/mutant_vs_wt_with_replicates.verbose;

### 5.7) Antisense transcription